# PTSD Mouse Data 

The total data consist of two data sets, measuring responses from 20 different mice at 4 timepoints each: Baseline, Pre-fear exposure, immediately after the Fear exposure, and 9 days later (D9). 10 of these mice are seratonin transporter knockouts (KO) and 10 are wildtype (WT). 

The data are: 
- 79 MRIs. The pre-fear, fear, and D9 images are MN(II) enhanced MRIs (MEMRI). The MEMRI images are used to measure neuronal functioning - (when neurons are active, their uptake of MN(II) is increased?). The baseline images, which are regular fMRI, are 
- 79 measurements of percent time spent in the light by the mice. 

In both data sets, the KO_04_D9 datapoint is missing (hence 79 instead of 80). 

In [2]:
# Import needed packages for analysis
import os

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import nibabel as nib
from scipy.stats import pearsonr
from scipy.stats import kendalltau


import cfl.util.brain_util as BU
import cfl.util.brain_vis as BV
import cfl.util.fear_mice_functions as fm 

mri_dir, mri_dims, affine, dir_labels = fm.get_global_values()

# load response data 
Y = pd.read_pickle('Y.pkl')

# MRI Data 
Some facts about the data: 
- the dimensions of the brain box in each image are (124, 200, 82) (2,033,600 voxels total)
- the images are originally in RPS orientation. We flip them to RAS orientation because then they have the same alignment as some other MRIs we've been looking at 

In [3]:
behav_csv = 'PTSD_Data_Share\Behavior_data\PTSD_PerLight.csv'
mri_dir = 'PTSD_Data_Share\MEMRI_data'

In [4]:
# The images start out in RPS orientation
nib_loaded_img = nib.load(os.path.join(mri_dir, "PTSD_KO_03_BL.nii"))
print(nib.orientations.aff2axcodes(nib_loaded_img.affine))

# load one image to check out its dimensions
img = BU.load_brain(os.path.join(mri_dir, "PTSD_KO_03_BL.nii"), ori='RPS')
mri_dims = img.shape
print(mri_dims)

('R', 'P', 'S')
(124, 200, 82)


In [5]:
# load all the images in RPS orientation 
X, Y_unused = BU.load_data(mri_dir, behav_csv, mri_dims, ori='RPS')

In [6]:
#check all images are the same shape 
for fp in os.listdir(mri_dir):
    full_path = os.path.join(mri_dir, fp)
    brain = BU.load_brain(full_path)
    assert brain.shape ==  mri_dims
    #print(brain.shape)

# Load Masks
We were given two masks to fit the the MRIs, a linearly aligned- and non-linearly aligned mask. These masks tell which voxels in the image are part of the brain vs which are empty space. 

The difference between the non-linear and linear mask (from an email from Taylor): "The non-linearly aligned mask is better aligned to the data, however the non-linear "warping" creates artifacts near the surface of the brain that may result in grabbing more undesired non-brain voxels when masking. The linearly aligned mask may not align to the surface of the brain as well and may miss or cut out brain tissue, but may not grab as many non-brain voxels. 

The non-linear mask leaves 531,632 voxels and the linear mask 482,793 voxels unmasked. 

In [7]:
# load the non-linear mask template
nl_mask_path = os.path.join('PTSD_Data_Share/templates\MuseTemplate_nonlinear_mask.nii')
nl_mask = BU.load_brain(nl_mask_path, ori='RPS')
nolin_mask_vec = BU.flatten(nl_mask)

# load the linear mask template
l_mask_path = os.path.join('PTSD_Data_Share/templates\MuseTemplate_linear_mask.nii')
l_mask = BU.load_brain(l_mask_path, ori='RPS')
lin_mask_vec = BU.flatten(l_mask)

In [8]:
# both masks are binary arrays, containing 1s where there is brain and 0s where there is not 
print(np.unique(nl_mask))
print(np.unique(l_mask))

[0. 1.]
[0. 1.]


In [9]:
print(nl_mask.shape == mri_dims) #non-linear mask has same dims as other MRI images
print(l_mask.shape == mri_dims) #linear mask has same dims as other MRI images

True
True


In [10]:
# how many voxels are unmasked? 
print("Non-linear mask: ", np.sum(nolin_mask_vec==1))
print("Linear mask: ", np.sum(lin_mask_vec==1))

Non-linear mask:  531632
Linear mask:  482793



When we visualize the two masks side-by-side on an MRI below, we indeed see that the non-linear mask has a bumpier edge, with more little islands of disconnected MRI image, while the linear mask has a smoother edge but cuts off some brain tissue. For now, we are using only the non-linearly aligned mask.

In [21]:
# create a copy of one image with each mask already applied
nl_masked_brain = X[0].copy()
nl_masked_brain[BU.flatten(nl_mask)==0] = np.nan

l_masked_brain = X[0].copy()
l_masked_brain[BU.flatten(l_mask)==0] = np.nan

# generate interactive plots
BV.plot_interactive_panels(np.vstack((nl_masked_brain, l_masked_brain)), mri_dims, nolin_mask_vec, figsize=(12, 3), std_scale='std', dir_labels=dir_labels, column_titles=["Non-linear Mask", "Linear Mask"])


interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

# Click through a bunch of the mice 

The below images compare, side-by-side, MRIs from multiple mice from the same timepoint. Since the first 5 brains in the list were selected, these are all MRIs from KO mice. 

In [12]:
timepoints_dir = fm.timepoint_indices_dir(Y)

# # Show first 5 baseline images 
# BV.plot_interactive_panels(X[timepoints_dir["BL"][:5]], mri_dims, nolin_mask_vec, figsize=(15, 5), colormap="seismic", dir_labels=dir_labels, column_titles=["Mouse 1", "Mouse 2", "Mouse 3", "Mouse 4", "Mouse 5"])

Some observations: 
Top (Saggital) Image 
- around slices 32-39, pieces of brain appear and disappear irregularly
- slice 39-40: lot more white in second image than others (and again around 85-89)
- 62 - dot of high activation, high for all except fourth

Middle (Coronal) Image 
- 20-40: noticeable difference in brightness of images (esp bt 1st and 3rd)
- higher brightness in 2 noticeable at 52-54

Botton (Horizontal) Image 
- the lower brightness of 4th brain is again visible (note high activation regions at slices ~16-36) 
- higher brightness of 2nd brain visible around slice ~46 


Do the white dots of high activity (ie those in observations above, also coronal slice 64) have a known meaning? 

In [13]:
# # Show first 5 prefear images 
# BV.plot_interactive_panels(X[timepoints_dir["PreF"][:5]], mri_dims, nolin_mask_vec, figsize=(15, 5), colormap="seismic", dir_labels=dir_labels, step=1)

3rd brain brightest, then 4th brain, least bright is 1st

Top (Saggital) Image 
- 39-40: bright spots (same as in baseline images)


Coronal 
- around slice 30-44: the circles in the brain kind of appear differently in different images - the brains may not be perfectly aligned ? 
- 54: bright spot 
- 68: bright spots 
- 80: really bright spots 
- 124: really bright spots


Horizontal
- lots of artifacts visible at beginning
- 17: really bright spots (brainstem connection to brain?)
- some interesting structures toward the end 


- which images are generally brightest/darkest vary between images

In [14]:
# # Show first 5 fear images 
# BV.plot_interactive_panels(X[timepoints_dir["Fear"][:5]], mri_dims, nolin_mask_vec, figsize=(15, 5), colormap="seismic", dir_labels=dir_labels, step=1)

Saggital image 
- 39: bright spot
- 54: tiny bright spot at the bottom of the brain 
- 62, 63, 64: abrupt shifts in where the bright/dark patterns are (this is present in the earlier images too but it's clearer in these)

Coronal 
- 30+ : the circles that seemed somewhat misaligned in the pre-fear images are not so in this set 
- 71: really bright spots 
- 77: really bright spots 


Horistonal 
- 16 bright spots 
- 56: on 2nd and 5th brains, the bright teardrop-shaped spots look a little asymetrical (left larger than right)



- All of these brains look generally consistent in brightness


These images generally look brighter, but I think that's just because the brightest brightest activation for this set of images (>25000) is lower than for BL or Pre-F (>30000)

In [15]:
# # Show first 5 D9 images 
# BV.plot_interactive_panels(X[timepoints_dir["D9"][:5]], mri_dims, nolin_mask_vec, figsize=(15, 5), colormap="seismic", dir_labels=dir_labels, step=1)

# Baseline Adjustment

In this section, images with the baseline MRI subtracted off are shown. 

## Baseline-adjusted Images for an individual mouse 



In [22]:
# plot to show the baseline-adjusted images for an individual mouse 
mouse_id = 7

# get image for that mouse at baseline 
baseline_index = Y.loc[(Y.ID==mouse_id) & (Y.Timepoint=="BL")].index[0]
baseline_image = X[baseline_index]

# get other origianl images for mouse
prefear_index = Y.loc[(Y.ID==mouse_id) & (Y.Timepoint=="PreF")].index[0]
postfear_index = Y.loc[(Y.ID==mouse_id) & (Y.Timepoint=="Fear")].index[0]
d9_index = Y.loc[(Y.ID==mouse_id) & (Y.Timepoint=="D9")].index[0]

#examine a single baseline adjusted image (individual images)
all_ims = np.zeros((3, np.prod(mri_dims)))
for i, timepoint in enumerate(["PreF", "Fear", "D9"]): 
    adj_im = fm.remove_baseline(X, Y, mouse_id, timepoint)
    adj_im = BU.flatten(adj_im)
    all_ims[i] += adj_im

# BV.plot_interactive_panels(np.vstack((baseline_image,  X[prefear_index], all_ims[0],  X[postfear_index], all_ims[1])), mri_dims, nolin_mask_vec, figsize=(17, 5), dir_labels=dir_labels, column_titles=["Baseline", "Pre-fear", "Adj Pre-fear", "Fear", "Adj Fear"])


interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

In [17]:
# BV.plot_interactive_panels(np.vstack((baseline_image, X[d9_index], all_ims[2])), mri_dims, nolin_mask_vec, figsize=(12, 5), dir_labels=dir_labels, column_titles=["Baseline", "D9", "Adj D9"])


## Heatmap of each timepoint, adjusted by baseline intensity 

Heatmap = mean value for each timepoint across all MRIs 

In [18]:
HM_dir = {'PreF': np.zeros(np.prod(mri_dims)), "Fear": np.zeros(np.prod(mri_dims)), 'D9': np.zeros(np.prod(mri_dims))}
all_mouse_indices = pd.unique(Y["ID"])


for timepoint in HM_dir:
    # for each mouse....
    for mouse_id in all_mouse_indices:
        # get the image associated with that timepoint, subtract off the baseline
        if mouse_id != 1 or timepoint != 'D9': #don't do this for the missing value
            adjusted_mri = fm.remove_baseline(X, Y, mouse_id, timepoint)
            # add the result to the correct heatmap entry 
            HM_dir[timepoint] += BU.flatten(adjusted_mri)
    # divide to get the average 
    n = len(timepoints_dir[timepoint])
    np.divide(HM_dir[timepoint], n) 

In [19]:
BV.plot_interactive_panels(np.vstack(list(HM_dir.values())), mri_dims, nolin_mask_vec, figsize=(12, 3), dir_labels=dir_labels, column_titles=list(HM_dir.keys()))


interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…